&nbsp;

# 15. Pré-traitement numérique pour SciML

---

&nbsp;

## 1. Introduction

Ce notebook vise à préparer l’état réduit issu de l’analyse EOF afin qu’il soit compatible avec l’apprentissage de modèles dynamiques SciML, cela en garantissant la stabilité numérique, l’absence de fuite d’information et la cohérence temporelle. On peut le réduire à une question centrale.

&nbsp;

> #### Est-ce mon état réduit est numériquement sain pour apprendre une dynamique ?

L’état réduit est constitué des $K$ premiers coefficients EOF (~150 modes dans le NetCDF), stockés sous la forme d’une série temporelle multivariée.

Ce que l'on connaît des modèles dynamique SciML, c'est leur tendance à être sensible aux échelles. Nous l'apprenons de cette source Application of Reduced-Order Models for Temporal Multiscale Representations in the Prediction of Dynamical Systems (https://arxiv.org/html/2510.18925v1) après une recherche avec le mot-clé "Model Reduction".

Globalement, ce que l'on en retire, c'est que ce type de modèle est conçu pour apprendre une loi d'évolution temporelle. Si l'état réduit "contient" des composantes associées à diverses échelles temporelles, le modèle appliqué doit réussir à différencier ces échelles au risque de sur-lisser ou ne modèliser qu'une seule échelle.

Dans un système réel (comme la SST), il existe des interactions entre échelles lentes et rapides (ou un spectre de K échelles). Les approches dites "naïves" peinent à capturer ces comportements qui opèrent simultanément sur le même jeu de données. 

Chaque mode (ou degré de liberté) peut donc évoluer à son propre rythme, et dans un système tel à plusieurs échelles, il nous faut au plus simplifier ce travail de différenciation pour que la formulation proposée par la modélisation dynamique SciML reproduise au plus fidèlement les signatures lentes/rapides et leurs couplages(ou le fait que l'évolution d’une composante du système dépend de l’état d’une ou plusieurs autres composantes). Exemple : un gradient thermique large échelle (EOF 1) peut conditionner l’intensité des fronts côtiers (EOF 3).

&nbsp;

---

&nbsp;

## 2. Normalisation et mise à l'échelle

> #### Pourquoi faire cela ?

La normalisation et la mise à l'échelle sont une réponse simple et efficace au problème de différencition d'échelles exprimée ci-dessus. On veut éviter qu'un mode ne domine trop numériquement. Mathématiquement cela se traduit par une normalisation par l'écart-type (ou scaling), comme suit :

$$
\tilde{a}_k(t) = \frac{a_k(t)}{\sigma_k}
$$
où :
- $\tilde{a}_k(t)$ : état réduit $k$ normalisé de sortie
- $a_k(t)$ : état réduit $k$ d'entrée 
- $\sigma_k$ : écart-type des PCs

En effet, diviser par l'écart-type ajuste notre état réduit de manière à affaiblir les PCs qui dominent et renforcer ceux qui sont dominés.

&nbsp;

In [6]:
import xarray as xr
import numpy as np

dsReducedState = xr.open_dataset("data/processed/sstReducedStateCOPERNICUS20102019.nc")

PCs = dsReducedState["PCs"].values

time = dsReducedState["time"] # Time splitting

trainMask = time < np.datetime64("2018-01-01") # We constrain training data to pre-2018

PCsTrain = dsReducedState["PCs"].sel(time=trainMask).values
PCsVal   = dsReducedState["PCs"].sel(time=~trainMask).values

stdTrain  = PCsTrain.std(axis=0)

PCsTrainScaled = PCsTrain / stdTrain
PCsValScaled   = PCsVal / stdTrain



&nbsp;

On revérifie que le centrage est correct pour garantir l'absence de biais constant dans la dynamique que l'on souhaite apprendre. On constate la moyenne de chaque PC est proche de 0 (de l'ordre du dix-millième au cent-milliardième selon le PC). 

C'est cette vérification qui justifie que nous n'ayons pas besoin de soustraire la moyenneau numérateur de la formule de normalisation énoncée.

&nbsp;

> #### Pourquoi normaliser uniquement sur les données d'entraînement ?

On sait que notre état réduit dépend désormais uniquement du temps, car on travaille bien avec $a_k(t)$.

On a expliqué pourquoi nous voulions normaliser les PCs, mais si nous calculons l'écart-type avec les PCs pour $t$ allant de 2010 à 2019 et que par la suite nous entrainons notre modèle sur une part des données (disons 01/01/2018). Un biais s'immisce dans notre apprentissage car la normalisation est réalisé avec une valeur qui incorpore une part de l'information future (l'écart-type). 

On appelle cela le biais d'anticipation, c'est-à-dire entraîner un modèle avec des paramètres qui capture une part de l'explicabilité future des valeurs que l'on cherche à prédire. L'apprentissage d'un modèle avec un tel biais aura tendance à sur-évaluer la précision de prédiction réelle du modèle à l'évaluation sur les données de tests. Dans ce cas on prédirait "mieux" les anomalies de SST de 2018-2019 mais moins bien que l'on le pourrait pour les années qui suivent.

Cela nous oblige dès maintenant à traiter la question du *splitting* des données d'entraînement et de validation.

In [5]:
# We recheck if scaling worked as intended
print(np.mean(PCsTrainScaled.mean(axis=0))) # ≈ 0
print(np.mean(PCsTrainScaled.std(axis=0))) # ≈ 1

0.0034608492
1.0


Nous nous assurons toujours que ces mesures restent proches des valeurs de référence voulues sous peine d'une exposition à des instabilités ou d'apprendre une tendance parasite lors de l'entraînement du modèle dynamique SciML.

On rappelle également que le pas de temps est régulier (voir Notebook 02), c'est important dans la mesure où les modèles SciML supposent une dynamique bien définie concernant la discrétisation temporelle (un pas de temps régulier pour garantir la validité de la formulation dynamique).

&nbsp;

---

&nbsp;

## 3. Construction du conteneur avant SciML


In [10]:
timeTrain = time[trainMask]
timeVal   = time[~trainMask]

dt = float((timeTrain[1] - timeTrain[0]) / np.timedelta64(1, "D")) # We precise float to avoid conflict between xarray and numpy types

tTrain = (np.arange(len(timeTrain)) * dt).astype(np.float32)
tVal   = (np.arange(len(timeVal)) * dt).astype(np.float32)

dataPrepared = {
    "PCsTrain": PCsTrainScaled,
    "PCsVal": PCsValScaled,
    "tTrain": tTrain,
    "tVal": tVal,
    "std": stdTrain,
}

# Saving prepared data
np.savez("data/processed/sstReducedStateCOPERNICUS20102019Prepared.npz", **dataPrepared)